In [1]:
import ast
from pandas.api.types import CategoricalDtype
import ipyparallel
import librosa
import numpy as np
import os
import pandas as pd
import pywt
from scipy.stats import skew

In [2]:
def remote_imports():
    import ast
    from pandas.api.types import CategoricalDtype
    import ipyparallel
    import librosa
    import numpy as np
    import os
    import pandas as pd
    import pywt
    from scipy.stats import skew
    
def load(filepath):
    """
    This method was adapted from the FMA: A Dataset For Music Analysis.
    """
    filename = os.path.basename(filepath)

    if 'features' in filename:
        return pd.read_csv(filepath, index_col=0, header=[0, 1, 2])

    if 'echonest' in filename:
        return pd.read_csv(filepath, index_col=0, header=[0, 1, 2])

    if 'genres' in filename:
        return pd.read_csv(filepath, index_col=None)

    if 'tracks' in filename:
        tracks = pd.read_csv(filepath, index_col=0, header=[0, 1])

        COLUMNS = [('track', 'tags'), ('album', 'tags'), ('artist', 'tags'),
                   ('track', 'genres'), ('track', 'genres_all')]
        for column in COLUMNS:
            tracks[column] = tracks[column].map(ast.literal_eval)

        COLUMNS = [('track', 'date_created'), ('track', 'date_recorded'),
                   ('album', 'date_created'), ('album', 'date_released'),
                   ('artist', 'date_created'), ('artist', 'active_year_begin'),
                   ('artist', 'active_year_end')]
        for column in COLUMNS:
            tracks[column] = pd.to_datetime(tracks[column])

        SUBSETS = ('small', 'medium', 'large')
        tracks['set', 'subset'] = tracks['set', 'subset'].astype(
                CategoricalDtype(categories=SUBSETS, ordered=True))

        COLUMNS = [('track', 'genre_top'), ('track', 'license'),
                   ('album', 'type'), ('album', 'information'),
                   ('artist', 'bio')]
        for column in COLUMNS:
            tracks[column] = tracks[column].astype('category')

        return tracks
        
def stats(feature):
    return {
        'mean': np.mean(feature), 
        'median': np.median(feature), 
        'std': np.std(feature), 
        'var': np.var(feature)
    }

def extra_stats(feature):
    return {
        'sb_energy': np.mean(np.abs(feature)),
        'skewness': skew(feature)
    }

In [3]:
# constants
MOUNTED_DATASET_PATH = '/home/macbookretina/s3-bucket'
LOCAL_MOUNTED_DATASET_PATH = '/Users/macbookretina/Desktop/mount-s3-bucket'
SAMPLE_FILE = '/Users/macbookretina/blues.00042.wav'
GENRES = ['hiphop', 'rock', 'pop']

In [4]:
# store for all features to be extracted except log-mel and mel-spectogram.
dataframe = pd.DataFrame({
    'genre_label': [],
    'data_source': [],
    
    'mean_spec_centroid': [],
    'median_spec_centroid': [],
    'std_spec_centroid': [],
    'var_spec_centroid': [],
    
    'mean_spec_rolloff': [],
    'median_spec_rolloff': [],
    'std_spec_rolloff': [],
    'var_spec_rolloff': [],
    
    'mean_zcr': [],
    'median_zcr': [],
    'std_zcr': [],
    'var_zcr': [],
    
    'mean_spec_bw': [],
    'median_spec_bw': [],
    'std_spec_bw': [],
    'var_spec_bw': [],
    
    'mean_spec_contrast_1': [],
    'median_spec_contrast_1': [],
    'std_spec_contrast_1': [],
    'var_spec_contrast_1': [],
    
    'mean_spec_contrast_2': [],
    'median_spec_contrast_2': [],
    'std_spec_contrast_2': [],
    'var_spec_contrast_2': [],
    
    'mean_spec_contrast_3': [],
    'median_spec_contrast_3': [],
    'std_spec_contrast_3': [],
    'var_spec_contrast_3': [],
    
    'mean_spec_contrast_4': [],
    'median_spec_contrast_4': [],
    'std_spec_contrast_4': [],
    'var_spec_contrast_4': [],
    
    'mean_spec_contrast_5': [],
    'median_spec_contrast_5': [],
    'std_spec_contrast_5': [],
    'var_spec_contrast_5': [],
    
    'mean_spec_contrast_6': [],
    'median_spec_contrast_6': [],
    'std_spec_contrast_6': [],
    'var_spec_contrast_6': [],
    
    'mean_spec_contrast_7': [],
    'median_spec_contrast_7': [],
    'std_spec_contrast_7': [],
    'var_spec_contrast_7': [],
    
    'mean_mfcc_1': [],
    'median_mfcc_1': [],
    'std_mfcc_1': [],
    'var_mfcc_1': [],
    
    'mean_mfcc_2': [],
    'median_mfcc_2': [],
    'std_mfcc_2': [],
    'var_mfcc_2': [],
    
    'mean_mfcc_3': [],
    'median_mfcc_3': [],
    'std_mfcc_3': [],
    'var_mfcc_3': [],
    
    'mean_mfcc_4': [],
    'median_mfcc_4': [],
    'std_mfcc_4': [],
    'var_mfcc_4': [],
    
    'mean_mfcc_5': [],
    'median_mfcc_5': [],
    'std_mfcc_5': [],
    'var_mfcc_5': [],
    
    'mean_mfcc_6': [],
    'median_mfcc_6': [],
    'std_mfcc_6': [],
    'var_mfcc_6': [],
    
    'mean_mfcc_7': [],
    'median_mfcc_7': [],
    'std_mfcc_7': [],
    'var_mfcc_7': [],
    
    'mean_mfcc_8': [],
    'median_mfcc_8': [],
    'std_mfcc_8': [],
    'var_mfcc_8': [],
    
    'mean_mfcc_9': [],
    'median_mfcc_9': [],
    'std_mfcc_9': [],
    'var_mfcc_9': [],
    
    'mean_mfcc_10': [],
    'median_mfcc_10': [],
    'std_mfcc_10': [],
    'var_mfcc_10': [],
    
    'mean_mfcc_11': [],
    'median_mfcc_11': [],
    'std_mfcc_11': [],
    'var_mfcc_11': [],
    
    'mean_mfcc_12': [],
    'median_mfcc_12': [],
    'std_mfcc_12': [],
    'var_mfcc_12': [],
    
    'mean_mfcc_13': [],
    'median_mfcc_13': [],
    'std_mfcc_13': [],
    'var_mfcc_13': [],
    
    'lpc_1': [],
    'lpc_2': [],
    'lpc_3': [],
    'lpc_4': [],
    
    'tempo': [],
    
    'mean_beats': [],
    'median_beats': [],
    'std_beats': [],
    'var_beats': [],
    
    'mean_beats_timestamp': [],
    'median_beats_timestamp': [],
    'std_beats_timestamp': [],
    'var_beats_timestamp': [],
    
    'mean_db4_cA4': [],
    'median_db4_cA4': [],
    'std_db4_cA4': [],
    'var_db4_cA4': [],
    'sb_energy_db4_cA4': [],
    'skewness_db4_cA4': [],
    
    'mean_db4_cD4': [],
    'median_db4_cD4': [],
    'std_db4_cD4': [],
    'var_db4_cD4': [],
    'sb_energy_db4_cD4': [],
    'skewness_db4_cD4': [],
    
    'mean_db4_cD3': [],
    'median_db4_cD3': [],
    'std_db4_cD3': [],
    'var_db4_cD3': [],
    'sb_energy_db4_cD3': [],
    'skewness_db4_cD3': [],
    
    'mean_db4_cD2': [],
    'median_db4_cD2': [],
    'std_db4_cD2': [],
    'var_db4_cD2': [],
    'sb_energy_db4_cD2': [],
    'skewness_db4_cD2': [],
    
    'mean_db4_cD1': [],
    'median_db4_cD1': [],
    'std_db4_cD1': [],
    'var_db4_cD1': [],
    'sb_energy_db4_cD1': [],
    'skewness_db4_cD1': [],
    
    'mean_db5_cA4': [],
    'median_db5_cA4': [],
    'std_db5_cA4': [],
    'var_db5_cA4': [],
    'sb_energy_db5_cA4': [],
    'skewness_db5_cA4': [],
    
    'mean_db5_cD4': [],
    'median_db5_cD4': [],
    'std_db5_cD4': [],
    'var_db5_cD4': [],
    'sb_energy_db5_cD4': [],
    'skewness_db5_cD4': [],
    
    'mean_db5_cD3': [],
    'median_db5_cD3': [],
    'std_db5_cD3': [],
    'var_db5_cD3': [],
    'sb_energy_db5_cD3': [],
    'skewness_db5_cD3': [],
    
    'mean_db5_cD2': [],
    'median_db5_cD2': [],
    'std_db5_cD2': [],
    'var_db5_cD2': [],
    'sb_energy_db5_cD2': [],
    'skewness_db5_cD2': [],
    
    'mean_db5_cD1': [],
    'median_db5_cD1': [],
    'std_db5_cD1': [],
    'var_db5_cD1': [],
    'sb_energy_db5_cD1': [],
    'skewness_db5_cD1': [],
    
    'mean_db8_cA7': [],
    'median_db8_cA7': [],
    'std_db8_cA7': [],
    'var_db8_cA7': [],
    'sb_energy_db8_cA7': [],
    'skewness_db8_cA7': [],
    
    'mean_db8_cD7': [],
    'median_db8_cD7': [],
    'std_db8_cD7': [],
    'var_db8_cD7': [],
    'sb_energy_db8_cD7': [],
    'skewness_db8_cD7': [],
    
    'mean_db8_cD6': [],
    'median_db8_cD6': [],
    'std_db8_cD6': [],
    'var_db8_cD6': [],
    'sb_energy_db8_cD6': [],
    'skewness_db8_cD6': [],
    
    'mean_db8_cD5': [],
    'median_db8_cD5': [],
    'std_db8_cD5': [],
    'var_db8_cD5': [],
    'sb_energy_db8_cD5': [],
    'skewness_db8_cD5': [],
    
    'mean_db8_cD4': [],
    'median_db8_cD4': [],
    'std_db8_cD4': [],
    'var_db8_cD4': [],
    'sb_energy_db8_cD4': [],
    'skewness_db8_cD4': [],
    
    'mean_db8_cD3': [],
    'median_db8_cD3': [],
    'std_db8_cD3': [],
    'var_db8_cD3': [],
    'sb_energy_db8_cD3': [],
    'skewness_db8_cD3': [],
    
    'mean_db8_cD2': [],
    'median_db8_cD2': [],
    'std_db8_cD2': [],
    'var_db8_cD2': [],
    'sb_energy_db8_cD2': [],
    'skewness_db8_cD2': [],
    
    'mean_db8_cD1': [],
    'median_db8_cD1': [],
    'std_db8_cD1': [],
    'var_db8_cD1': [],
    'sb_energy_db8_cD1': [],
    'skewness_db8_cD1': [],
    
})

In [5]:
def extract_audio_features(dataframe, file, genre_label, data_source):
    '''
    This function takes a datafame, an audio file (check librosa for acceptable formats),
    genre label, and datasource. It extract features from the audio and returns the dataframe with 
    the new row appended.
    
    Timbral, rhythmic, and wavelet features are extracted excluding log-mel and mel-spectogram.
    
    Parameters:
    dataframe (pd.Dataframe): Dataframe to update with new row.
    file (File or str): an audio file or file path.
    genre_label (str): audio genre label
    data_source (str): fma or gtzan
    '''
    # get sample rate of audio file
    sample_rate = librosa.core.get_samplerate(file.path)

    # load audio file as time series
    time_series, _ = librosa.core.load(file.path, sample_rate)


    # compute timbral features
    # compute spectral centroid
    spec_centroid = librosa.feature.spectral_centroid(time_series, sample_rate)
    stats_spec_centroid = stats(spec_centroid)

    # compute spectral roll-off
    spec_rolloff = librosa.feature.spectral_rolloff(time_series, sample_rate)
    stats_spec_rolloff = stats(spec_rolloff)

    # compute zero crossing rate
    zcr = librosa.feature.zero_crossing_rate(time_series)
    stats_zcr = stats(zcr)

    # compute spectral bandwidth
    spec_bw = librosa.feature.spectral_bandwidth(time_series, sample_rate)
    stats_spec_bw = stats(spec_bw[0])

    # compute spectral contrast
    spec_contrast = librosa.feature.spectral_contrast(time_series, sample_rate)
    stats_spec_contrast_1 = stats(spec_contrast[0])
    stats_spec_contrast_2 = stats(spec_contrast[1])
    stats_spec_contrast_3 = stats(spec_contrast[2])
    stats_spec_contrast_4 = stats(spec_contrast[3])
    stats_spec_contrast_5 = stats(spec_contrast[4])
    stats_spec_contrast_6 = stats(spec_contrast[5])
    stats_spec_contrast_7 = stats(spec_contrast[6])

    # compute 13 mel-frequency cepstral coefficients
    mfcc = librosa.feature.mfcc(time_series, sample_rate, n_mfcc=13)
    stat_mfcc_1 = stats(mfcc[0])
    stat_mfcc_2 = stats(mfcc[1])
    stat_mfcc_3 = stats(mfcc[2])
    stat_mfcc_4 = stats(mfcc[3])
    stat_mfcc_5 = stats(mfcc[4])
    stat_mfcc_6 = stats(mfcc[5])
    stat_mfcc_7 = stats(mfcc[6])
    stat_mfcc_8 = stats(mfcc[7])
    stat_mfcc_9 = stats(mfcc[8])
    stat_mfcc_10 = stats(mfcc[9])
    stat_mfcc_11 = stats(mfcc[10])
    stat_mfcc_12 = stats(mfcc[11])
    stat_mfcc_13 = stats(mfcc[12])

    # compute 3rd order linear prediction coefficients
    lpc = librosa.lpc(time_series, 3)


    # compute rhythmic features
    # compute tempo & beats
    tempo, beats = librosa.beat.beat_track(time_series, sample_rate)
    stats_beats = stats(beats)

    # compute timestamps from beats
    beats_timestamp = librosa.frames_to_time(beats, sample_rate)
    stats_beats_timestamp = stats(beats_timestamp)


    # compute wavelet features
    # compute coefficients for Db4 at level 4 decomposition
    db4_coeffs = pywt.wavedec(time_series, 'db4', level=4)
    db4_cA4, db4_cD4, db4_cD3, db4_cD2, db4_cD1 = db4_coeffs
    stats_db4_cA4 = {**stats(db4_cA4), **extra_stats(db4_cA4)}
    stats_db4_cD4 = {**stats(db4_cD4), **extra_stats(db4_cD4)}
    stats_db4_cD3 = {**stats(db4_cD3), **extra_stats(db4_cD3)}
    stats_db4_cD2 = {**stats(db4_cD2), **extra_stats(db4_cD2)}
    stats_db4_cD1 = {**stats(db4_cD1), **extra_stats(db4_cD1)}

    # compute coefficients for Db5 at level 4 decomposition
    db5_coeffs = pywt.wavedec(time_series, 'db5', level=4)
    db5_cA4, db5_cD4, db5_cD3, db5_cD2, db5_cD1 = db5_coeffs
    stats_db5_cA4 = {**stats(db5_cA4), **extra_stats(db5_cA4)}
    stats_db5_cD4 = {**stats(db5_cD4), **extra_stats(db5_cD4)}
    stats_db5_cD3 = {**stats(db5_cD3), **extra_stats(db5_cD3)}
    stats_db5_cD2 = {**stats(db5_cD2), **extra_stats(db5_cD2)}
    stats_db5_cD1 = {**stats(db5_cD1), **extra_stats(db5_cD1)}

    # compute coefficients for Db8 at level 7 decomposition
    db8_coeffs = pywt.wavedec(time_series, 'db4', level=7)
    db8_cA7, db8_cD7, db8_cD6, db8_cD5, db8_cD4, db8_cD3, db8_cD2, db8_cD1 = db8_coeffs
    stats_db8_cA7 = {**stats(db8_cA7), **extra_stats(db8_cA7)}
    stats_db8_cD7 = {**stats(db8_cD7), **extra_stats(db8_cD7)}
    stats_db8_cD6 = {**stats(db8_cD6), **extra_stats(db8_cD6)}
    stats_db8_cD5 = {**stats(db8_cD5), **extra_stats(db8_cD5)}
    stats_db8_cD4 = {**stats(db8_cD4), **extra_stats(db8_cD4)}
    stats_db8_cD3 = {**stats(db8_cD3), **extra_stats(db8_cD3)}
    stats_db8_cD2 = {**stats(db8_cD2), **extra_stats(db8_cD2)}
    stats_db8_cD1 = {**stats(db8_cD1), **extra_stats(db8_cD1)}
    
    # create new row
    new_row = {
    'genre_label': genre_label,
    'data_source': data_source,
        
    'mean_spec_centroid': stats_spec_centroid['mean'],
    'median_spec_centroid': stats_spec_centroid['median'],
    'std_spec_centroid': stats_spec_centroid['std'],
    'var_spec_centroid': stats_spec_centroid['var'],
    
    'mean_spec_rolloff': stats_spec_rolloff['mean'],
    'median_spec_rolloff': stats_spec_rolloff['median'],
    'std_spec_rolloff': stats_spec_rolloff['std'],
    'var_spec_rolloff': stats_spec_rolloff['var'],
    
    'mean_zcr': stats_zcr['mean'],
    'median_zcr': stats_zcr['median'],
    'std_zcr': stats_zcr['std'],
    'var_zcr': stats_zcr['var'],
    
    'mean_spec_bw': stats_spec_bw['mean'],
    'median_spec_bw': stats_spec_bw['median'],
    'std_spec_bw': stats_spec_bw['std'],
    'var_spec_bw': stats_spec_bw['var'],
    
    'mean_spec_contrast_1': stats_spec_contrast_1['mean'],
    'median_spec_contrast_1': stats_spec_contrast_1['median'],
    'std_spec_contrast_1': stats_spec_contrast_1['std'],
    'var_spec_contrast_1': stats_spec_contrast_1['var'],
    
    'mean_spec_contrast_2': stats_spec_contrast_2['mean'],
    'median_spec_contrast_2': stats_spec_contrast_2['median'],
    'std_spec_contrast_2': stats_spec_contrast_2['std'],
    'var_spec_contrast_2': stats_spec_contrast_2['var'],
    
    'mean_spec_contrast_3': stats_spec_contrast_3['mean'],
    'median_spec_contrast_3': stats_spec_contrast_3['median'],
    'std_spec_contrast_3': stats_spec_contrast_3['std'],
    'var_spec_contrast_3': stats_spec_contrast_3['var'],
    
    'mean_spec_contrast_4': stats_spec_contrast_4['mean'],
    'median_spec_contrast_4': stats_spec_contrast_4['median'],
    'std_spec_contrast_4': stats_spec_contrast_4['std'],
    'var_spec_contrast_4': stats_spec_contrast_4['var'],
    
    'mean_spec_contrast_5': stats_spec_contrast_5['mean'],
    'median_spec_contrast_5': stats_spec_contrast_5['median'],
    'std_spec_contrast_5': stats_spec_contrast_5['std'],
    'var_spec_contrast_5': stats_spec_contrast_5['var'],
    
    'mean_spec_contrast_6': stats_spec_contrast_6['mean'],
    'median_spec_contrast_6': stats_spec_contrast_6['median'],
    'std_spec_contrast_6': stats_spec_contrast_6['std'],
    'var_spec_contrast_6': stats_spec_contrast_6['var'],
    
    'mean_spec_contrast_7': stats_spec_contrast_7['mean'],
    'median_spec_contrast_7': stats_spec_contrast_7['median'],
    'std_spec_contrast_7': stats_spec_contrast_7['std'],
    'var_spec_contrast_7': stats_spec_contrast_7['var'],
    
    'mean_mfcc_1': stat_mfcc_1['mean'],
    'median_mfcc_1': stat_mfcc_1['median'],
    'std_mfcc_1': stat_mfcc_1['std'],
    'var_mfcc_1': stat_mfcc_1['var'],
    
    'mean_mfcc_2': stat_mfcc_2['mean'],
    'median_mfcc_2': stat_mfcc_2['median'],
    'std_mfcc_2': stat_mfcc_2['std'],
    'var_mfcc_2': stat_mfcc_2['var'],
    
    'mean_mfcc_3': stat_mfcc_3['mean'],
    'median_mfcc_3': stat_mfcc_3['median'],
    'std_mfcc_3': stat_mfcc_3['std'],
    'var_mfcc_3': stat_mfcc_3['var'],
    
    'mean_mfcc_4': stat_mfcc_4['mean'],
    'median_mfcc_4': stat_mfcc_4['median'],
    'std_mfcc_4': stat_mfcc_4['std'],
    'var_mfcc_4': stat_mfcc_4['var'],
    
    'mean_mfcc_5': stat_mfcc_5['mean'],
    'median_mfcc_5': stat_mfcc_5['median'],
    'std_mfcc_5': stat_mfcc_5['std'],
    'var_mfcc_5': stat_mfcc_5['var'],
    
    'mean_mfcc_6': stat_mfcc_6['mean'],
    'median_mfcc_6': stat_mfcc_6['median'],
    'std_mfcc_6': stat_mfcc_6['std'],
    'var_mfcc_6': stat_mfcc_6['var'],
    
    'mean_mfcc_7': stat_mfcc_7['mean'],
    'median_mfcc_7': stat_mfcc_7['median'],
    'std_mfcc_7': stat_mfcc_7['std'],
    'var_mfcc_7': stat_mfcc_7['var'],
    
    'mean_mfcc_8': stat_mfcc_8['mean'],
    'median_mfcc_8': stat_mfcc_8['median'],
    'std_mfcc_8': stat_mfcc_8['std'],
    'var_mfcc_8':stat_mfcc_8['var'],
    
    'mean_mfcc_9': stat_mfcc_9['mean'],
    'median_mfcc_9': stat_mfcc_9['median'],
    'std_mfcc_9': stat_mfcc_9['std'],
    'var_mfcc_9': stat_mfcc_9['var'],
    
    'mean_mfcc_10': stat_mfcc_10['mean'],
    'median_mfcc_10': stat_mfcc_10['median'],
    'std_mfcc_10': stat_mfcc_10['std'],
    'var_mfcc_10': stat_mfcc_10['var'],
    
    'mean_mfcc_11': stat_mfcc_11['mean'],
    'median_mfcc_11': stat_mfcc_11['median'],
    'std_mfcc_11': stat_mfcc_11['std'],
    'var_mfcc_11': stat_mfcc_11['var'],
    
    'mean_mfcc_12': stat_mfcc_12['mean'],
    'median_mfcc_12': stat_mfcc_12['median'],
    'std_mfcc_12': stat_mfcc_12['std'],
    'var_mfcc_12': stat_mfcc_12['var'],
    
    'mean_mfcc_13': stat_mfcc_13['mean'],
    'median_mfcc_13': stat_mfcc_13['median'],
    'std_mfcc_13': stat_mfcc_13['std'],
    'var_mfcc_13': stat_mfcc_13['var'],
    
    'lpc_1': lpc[0],
    'lpc_2': lpc[1],
    'lpc_3': lpc[2],
    'lpc_4': lpc[3],
    
    'tempo': tempo,
    
    'mean_beats': stats_beats['mean'],
    'median_beats': stats_beats['median'],
    'std_beats': stats_beats['std'],
    'var_beats': stats_beats['var'],
    
    'mean_beats_timestamp': stats_beats_timestamp['mean'],
    'median_beats_timestamp': stats_beats_timestamp['median'],
    'std_beats_timestamp': stats_beats_timestamp['std'],
    'var_beats_timestamp': stats_beats_timestamp['var'],
    
    'mean_db4_cA4': stats_db4_cA4['mean'],
    'median_db4_cA4': stats_db4_cA4['median'],
    'std_db4_cA4': stats_db4_cA4['std'],
    'var_db4_cA4': stats_db4_cA4['var'],
    'sb_energy_db4_cA4': stats_db4_cA4['sb_energy'],
    'skewness_db4_cA4': stats_db4_cA4['skewness'],
    
    'mean_db4_cD4': stats_db4_cD4['mean'],
    'median_db4_cD4': stats_db4_cD4['median'],
    'std_db4_cD4': stats_db4_cD4['std'],
    'var_db4_cD4': stats_db4_cD4['var'],
    'sb_energy_db4_cD4': stats_db4_cD4['sb_energy'],
    'skewness_db4_cD4': stats_db4_cD4['skewness'],
    
    'mean_db4_cD3': stats_db4_cD3['mean'],
    'median_db4_cD3': stats_db4_cD3['median'],
    'std_db4_cD3': stats_db4_cD3['std'],
    'var_db4_cD3': stats_db4_cD3['var'],
    'sb_energy_db4_cD3': stats_db4_cD3['sb_energy'],
    'skewness_db4_cD3': stats_db4_cD3['skewness'],
    
    'mean_db4_cD2': stats_db4_cD2['mean'],
    'median_db4_cD2': stats_db4_cD2['median'],
    'std_db4_cD2': stats_db4_cD2['std'],
    'var_db4_cD2': stats_db4_cD2['var'],
    'sb_energy_db4_cD2': stats_db4_cD2['sb_energy'],
    'skewness_db4_cD2': stats_db4_cD2['skewness'],
    
    'mean_db4_cD1': stats_db4_cD1['mean'],
    'median_db4_cD1': stats_db4_cD1['median'],
    'std_db4_cD1': stats_db4_cD1['std'],
    'var_db4_cD1': stats_db4_cD1['var'],
    'sb_energy_db4_cD1': stats_db4_cD1['sb_energy'],
    'skewness_db4_cD1': stats_db4_cD1['skewness'],
    
    'mean_db5_cA4': stats_db5_cA4['mean'],
    'median_db5_cA4': stats_db5_cA4['median'],
    'std_db5_cA4': stats_db5_cA4['std'],
    'var_db5_cA4': stats_db5_cA4['var'],
    'sb_energy_db5_cA4': stats_db5_cA4['sb_energy'],
    'skewness_db5_cA4': stats_db5_cA4['skewness'],
    
    'mean_db5_cD4': stats_db5_cD4['mean'],
    'median_db5_cD4': stats_db5_cD4['median'],
    'std_db5_cD4': stats_db5_cD4['std'],
    'var_db5_cD4': stats_db5_cD4['var'],
    'sb_energy_db5_cD4': stats_db5_cD4['sb_energy'],
    'skewness_db5_cD4': stats_db5_cD4['skewness'],
    
    'mean_db5_cD3': stats_db5_cD3['mean'],
    'median_db5_cD3': stats_db5_cD3['median'],
    'std_db5_cD3': stats_db5_cD3['std'],
    'var_db5_cD3': stats_db5_cD3['var'],
    'sb_energy_db5_cD3': stats_db5_cD3['sb_energy'],
    'skewness_db5_cD3': stats_db5_cD3['skewness'],
    
    'mean_db5_cD2': stats_db5_cD2['mean'],
    'median_db5_cD2': stats_db5_cD2['median'],
    'std_db5_cD2': stats_db5_cD2['std'],
    'var_db5_cD2': stats_db5_cD2['var'],
    'sb_energy_db5_cD2': stats_db5_cD2['sb_energy'],
    'skewness_db5_cD2': stats_db5_cD2['skewness'],
    
    'mean_db5_cD1': stats_db5_cD1['mean'],
    'median_db5_cD1': stats_db5_cD1['median'],
    'std_db5_cD1': stats_db5_cD1['std'],
    'var_db5_cD1': stats_db5_cD1['var'],
    'sb_energy_db5_cD1': stats_db5_cD1['sb_energy'],
    'skewness_db5_cD1': stats_db5_cD1['skewness'],
    
    'mean_db8_cA7': stats_db8_cA7['mean'],
    'median_db8_cA7': stats_db8_cA7['median'],
    'std_db8_cA7': stats_db8_cA7['std'],
    'var_db8_cA7': stats_db8_cA7['var'],
    'sb_energy_db8_cA7': stats_db8_cA7['sb_energy'],
    'skewness_db8_cA7': stats_db8_cA7['skewness'],
    
    'mean_db8_cD7': stats_db8_cD7['mean'],
    'median_db8_cD7': stats_db8_cD7['median'],
    'std_db8_cD7': stats_db8_cD7['std'],
    'var_db8_cD7': stats_db8_cD7['var'],
    'sb_energy_db8_cD7': stats_db8_cD7['sb_energy'],
    'skewness_db8_cD7': stats_db8_cD7['skewness'],
    
    'mean_db8_cD6': stats_db8_cD6['mean'],
    'median_db8_cD6': stats_db8_cD6['median'],
    'std_db8_cD6': stats_db8_cD6['std'],
    'var_db8_cD6': stats_db8_cD6['var'],
    'sb_energy_db8_cD6': stats_db8_cD6['sb_energy'],
    'skewness_db8_cD6': stats_db8_cD6['skewness'],
    
    'mean_db8_cD5': stats_db8_cD5['mean'],
    'median_db8_cD5': stats_db8_cD5['median'],
    'std_db8_cD5': stats_db8_cD5['std'],
    'var_db8_cD5': stats_db8_cD5['var'],
    'sb_energy_db8_cD5': stats_db8_cD5['sb_energy'],
    'skewness_db8_cD5': stats_db8_cD5['skewness'],
    
    'mean_db8_cD4': stats_db8_cD4['mean'],
    'median_db8_cD4': stats_db8_cD4['median'],
    'std_db8_cD4': stats_db8_cD4['std'],
    'var_db8_cD4': stats_db8_cD4['var'],
    'sb_energy_db8_cD4': stats_db8_cD4['sb_energy'],
    'skewness_db8_cD4': stats_db8_cD4['skewness'],
    
    'mean_db8_cD3': stats_db8_cD3['mean'],
    'median_db8_cD3': stats_db8_cD3['median'],
    'std_db8_cD3': stats_db8_cD3['std'],
    'var_db8_cD3': stats_db8_cD3['var'],
    'sb_energy_db8_cD3': stats_db8_cD3['sb_energy'],
    'skewness_db8_cD3': stats_db8_cD3['skewness'],
    
    'mean_db8_cD2': stats_db8_cD2['mean'],
    'median_db8_cD2': stats_db8_cD2['median'],
    'std_db8_cD2': stats_db8_cD2['std'],
    'var_db8_cD2': stats_db8_cD2['var'],
    'sb_energy_db8_cD2': stats_db8_cD2['sb_energy'],
    'skewness_db8_cD2': stats_db8_cD2['skewness'],
    
    'mean_db8_cD1': stats_db8_cD1['mean'],
    'median_db8_cD1': stats_db8_cD1['median'],
    'std_db8_cD1': stats_db8_cD1['std'],
    'var_db8_cD1': stats_db8_cD1['var'],
    'sb_energy_db8_cD1': stats_db8_cD1['sb_energy'],
    'skewness_db8_cD1': stats_db8_cD1['skewness'],
    }
    
    # append new row
    dataframe = dataframe.append(new_row, ignore_index=True)
    
    print('appended features extracted from ' + str(file.name) + ' with genre: ' + genre_label )
    
    return dataframe

In [6]:
# FMA
# collect track id and genres of tracks in the small subset.

print('collecting track id and genres of tracks in the small subset of fma dataset')

tracks = load(MOUNTED_DATASET_PATH + '/fma_metadata/tracks.csv')
fma_full = tracks[[('set', 'subset'), ('track', 'genre_top')]]
small_subset = fma_full[('set', 'subset')] == 'small'
fma_small = fma_full[small_subset]
fma_small = pd.DataFrame({
    'subset': fma_small[('set', 'subset')],
    'label': fma_small[('track', 'genre_top')]
})

print('done')

collecting track id and genres of tracks in the small subset of fma dataset
done


In [7]:
# attach to a running cluster to notebook
cluster = ipyparallel.Client()

# Print profile name and process id numbers
print('profile:', cluster.profile)
print("IDs:", cluster.ids)

profile: default
IDs: [0, 1, 2, 3, 4, 5, 6, 7]


In [8]:
# split into tow set of 4 engines each and get DirectView object of each set.
direct_views_first_4_engines = cluster[:4]
direct_views_last_4_engines = cluster[5:]

In [9]:
# sync import for each DirectView object
with direct_views_first_4_engines.sync_imports():
    remote_imports()
    
with direct_views_last_4_engines.sync_imports():
    remote_imports()

importing ast on engine(s)
importing CategoricalDtype from pandas.api.types on engine(s)
importing ipyparallel on engine(s)
importing librosa on engine(s)
importing numpy on engine(s)
importing os on engine(s)
importing pandas on engine(s)
importing pywt on engine(s)
importing skew from scipy.stats on engine(s)
importing ast on engine(s)
importing CategoricalDtype from pandas.api.types on engine(s)
importing ipyparallel on engine(s)
importing librosa on engine(s)
importing numpy on engine(s)
importing os on engine(s)
importing pandas on engine(s)
importing pywt on engine(s)
importing skew from scipy.stats on engine(s)


In [10]:
# initialize shared resouces for each DirectView object
direct_views_first_4_engines.push({
    'MOUNTED_DATASET_PATH': MOUNTED_DATASET_PATH,
    'LOCAL_MOUNTED_DATASET_PATH': LOCAL_MOUNTED_DATASET_PATH,
    'SAMPLE_FILE': SAMPLE_FILE,
    'GENRES': GENRES,
    'extract_audio_features': extract_audio_features,
    'dataframe': dataframe,
    'fma_small': fma_small,
})

direct_views_last_4_engines.push({
    'MOUNTED_DATASET_PATH': MOUNTED_DATASET_PATH,
    'LOCAL_MOUNTED_DATASET_PATH': LOCAL_MOUNTED_DATASET_PATH,
    'SAMPLE_FILE': SAMPLE_FILE,
    'GENRES': GENRES,
    'extract_audio_features': extract_audio_features,
    'dataframe': dataframe,
    'fma_small': fma_small,
})

<AsyncResult: _push>

In [11]:
# activate DirectView object and set suffix
direct_views_first_4_engines.activate('_first')
direct_views_last_4_engines.activate('_last')

In [ ]:
# GTZAN
# extract features

print('extracting features from gtzan dataset')

for file in os.scandir(MOUNTED_DATASET_PATH + '/gtzan/wavfiles'):
    if file.is_file():
        genre_label = str(file.name).split('.')[0]
        if genre_label in GENRES:
            dataframe = extract_audio_features(dataframe, file, genre_label, 'gtzan')

print('done')

In [ ]:
# FMA
# extract features

print('extracting features from fma dataset')

for directory in os.scandir(MOUNTED_DATASET_PATH + '/fma_small'):
    if directory.is_dir():
        for file in os.scandir(directory.path):
            if file.is_file():
                track_id = int(file.name[:-4].lstrip('0'))
                # map track id to genre label
                genre_label = fma_small.at[track_id, 'label'].lower().replace('-', '')
                if genre_label in GENRES:
                    dataframe = extract_audio_features(dataframe, file, genre_label, 'fma')

print(done)

In [41]:
# check size of  log-mel / constant-Q transform in gtzan

# creating an empty list to store sizes in
sizes_1 = []

print('extracting log-mel from gtzan dataset')

for file in os.scandir(MOUNTED_DATASET_PATH + '/gtzan/wavfiles'):
    if file.is_file():
        
        # extract genre label
        genre_label = str(file.name).split('.')[0]
        
        if genre_label in GENRES:
            
            # get sample rate of audio file
            sample_rate = librosa.core.get_samplerate(file.path)

            # load audio file as time series
            time_series, _ = librosa.core.load(file.path, sample_rate)

            # compute cqt
            cqt = librosa.cqt(time_series, sample_rate)

            # convert from amplitude to decibels unit
            scaled_cqt = librosa.amplitude_to_db(cqt, ref=np.max) 

            # adding the size to the list
            sizes.append(scaled_cqt.shape)

# check if all sizes are the same
print('The sizes of all the log-mel in our data set are equal: ' +  str(len(set(sizes)) == 1))

# check the max size
print('The maximum size is: ' + str(max(sizes)))

extracting log-mel from gtzan dataset


/home/macbookretina/.local/lib/python3.5/site-packages/librosa/core/spectrum.py:1702: UserWarning: amplitude_to_db was called on complex input so phase information will be discarded. To suppress this warning, call amplitude_to_db(np.abs(S)) instead.
  warnings.warn('amplitude_to_db was called on complex input so phase '


The sizes of all the log-mel in our data set are equal: False
The maximum size is: (84, 1320)


In [42]:
# check size of mel-spectogram in gtzan

# creating an empty list to store sizes in
sizes_2 = []

print('extracting mel-spectogram from gtzan dataset')

for file in os.scandir(MOUNTED_DATASET_PATH + '/gtzan/wavfiles'):
    if file.is_file():
        
        # extract genre label
        genre_label = str(file.name).split('.')[0]
        
        if genre_label in GENRES:
            
            # get sample rate of audio file
            sample_rate = librosa.core.get_samplerate(file.path)

            # load audio file as time series
            time_series, _ = librosa.core.load(file.path, sample_rate)

            # compute spectogram
            mel_spect = librosa.feature.melspectrogram(time_series, sample_rate)

            # convert spectogram to decibels unit 
            scaled_mel_spect = librosa.power_to_db(mel_spect, ref=np.max)

            # adding the size to the list
            sizes.append(scaled_mel_spect.shape)
    
# check if all sizes are the same
print('The sizes of all the log-mel in our data set are equal: ' +  str(len(set(sizes)) == 1))

# check the max size
print('The maximum size is: ' + str(max(sizes)))

extracting mel-spectogram from gtzan dataset
The sizes of all the log-mel in our data set are equal: False
The maximum size is: (128, 1320)


In [16]:
%%px_first --targets 0 --noblock
# check size of  log-mel / constant-Q transform in fma
    
# creating an empty list to store sizes in
sizes = []

print('extracting log-mel from fma dataset')

for directory in os.scandir(MOUNTED_DATASET_PATH + '/fma_small'):
    if directory.is_dir():
        for file in os.scandir(directory.path):
            if file.is_file():

                # extract track id
                track_id = int(file.name[:-4].lstrip('0'))

                # map track id to genre label
                genre_label = fma_small.at[track_id, 'label'].lower().replace('-', '')

                if genre_label in GENRES:

                    # get sample rate of audio file
                    sample_rate = librosa.core.get_samplerate(str(file.path))

                    # load audio file as time series
                    time_series, _ = librosa.core.load(file.path, sample_rate)

                    # compute cqt
                    cqt = librosa.cqt(time_series, sample_rate)

                    # convert from amplitude to decibels unit
                    scaled_cqt = librosa.amplitude_to_db(cqt, ref=numpy.max) 

                    # adding the size to the list
                    sizes.append(scaled_cqt.shape)

# check if all sizes are the same
print('The sizes of all the log-mel in our data set are equal: ' +  str(len(set(sizes)) == 1))

# check the max size
print('The maximum size is: ' + str(max(sizes)))

<AsyncResult: execute>

In [17]:
%%px_last --targets 4 --noblock
# check size of  mel-spectogram  in fma

# creating an empty list to store sizes in
sizes = []

print('extracting mel-spectogram  from fma dataset')

for directory in os.scandir(MOUNTED_DATASET_PATH + '/fma_small'):
    if directory.is_dir():
        for file in os.scandir(directory.path):
            if file.is_file():

                # extract track id
                track_id = int(file.name[:-4].lstrip('0'))

                # map track id to genre label
                genre_label = fma_small.at[track_id, 'label'].lower().replace('-', '')

                if genre_label in GENRES:

                    # get sample rate of audio file
                    sample_rate = librosa.core.get_samplerate(file.path)

                    # load audio file as time series
                    time_series, _ = librosa.core.load(file.path, sample_rate)

                    # compute spectogram
                    mel_spect = librosa.feature.melspectrogram(time_series, sample_rate)

                    # convert spectogram to decibels unit 
                    scaled_mel_spect = librosa.power_to_db(mel_spect, ref=numpy.max)

                    # adding the size to the list
                    sizes.append(scaled_mel_spect.shape)                

# check if all sizes are the same
print('The sizes of all the log-mel in our data set are equal: ' +  str(len(set(sizes)) == 1))

# check the max size
print('The maximum size is: ' + str(max(sizes)))

<AsyncResult: execute>

In [ ]:
%pxresult_first

In [ ]:
%pxresult_last

In [ ]:
# extract log-mel / constant-Q transform in gtzan

# create an empty list to store extract feature and label
cqts = []
genre_labels_1 = []

print('extracting log-mel from gtzan dataset')

for file in os.scandir(MOUNTED_DATASET_PATH + '/gtzan/wavfiles'):
    if file.is_file():
        # extract genre label and append to list
        genre_label = str(file.name).split('.')[0]
        genre_labels_1.append(genre_label)
        
        if genre_label in GENRES:
        
            # get sample rate of audio file
            sample_rate = librosa.core.get_samplerate(file.path)

            # load audio file as time series
            time_series, _ = librosa.core.load(file.path, sample_rate)

            # compute cqt
            cqt = librosa.cqt(time_series, sample_rate)

            # convert from amplitude to decibels unit
            scaled_cqt = librosa.amplitude_to_db(cqt, ref=np.max) 

            # adjust the size to (84, 1320) as it is the max size
            if scaled_cqt.shape[1] != 1320:
                scaled_cqt.resize(84, 1320, refcheck=False)

            # flatten to fit into dataframe and add to the list
            scaled_cqt = scaled_cqt.flatten()
            cqts.append(scaled_cqt)
        
# convert the lists to arrays so it can be stacked
cqts = np.array(cqts)
genre_labels = np.array(genre_labels).reshape(1000, 1)

# create dataframe
cqt_df = pd.DataFrame(np.hstack((genre_labels, cqts)))    

In [ ]:
# extract mel-spectogram in gtzan

# create an empty list to store extract feature and label
mel_spects = []
genre_labels_2 = []

print('extracting mel-spectogram from gtzan dataset')

for file in os.scandir(MOUNTED_DATASET_PATH + '/gtzan/wavfiles'):
    if file.is_file():
        # extract genre label and append to list
        genre_label = str(file.name).split('.')[0]
        genre_labels_2.append(genre_label)
        
        if genre_label in GENRES:
        
            # get sample rate of audio file
            sample_rate = librosa.core.get_samplerate(file.path)

            # load audio file as time series
            time_series, _ = librosa.core.load(file.path, sample_rate)

            # compute spectogram
            mel_spect = librosa.feature.melspectrogram(time_series, sample_rate)

            # convert spectogram to decibels unit 
            scaled_mel_spect = librosa.power_to_db(mel_spect, ref=np.max)

            # adjust the size to (128, 1320) as it is the max size
            if scaled_mel_spect.shape[1] != 1320:
                scaled_mel_spect.resize(128, 1320, refcheck=False)

            # flatten to fit into dataframe and add to the list
            scaled_mel_spect = scaled_mel_spect.flatten()
            mel_spects.append(scaled_mel_spect)
        
# convert the lists to arrays so it can be stacked
mel_spects = np.array(mel_spects)
genre_labels = np.array(genre_labels).reshape(1000, 1)

# create dataframe
mel_spect_df = pd.DataFrame(np.hstack((genre_labels, mel_spects)))

In [12]:
%%px_first --targets 0 --noblock
print('start 1')
import time
time.sleep(10)
print(time.localtime())

<AsyncResult: execute>

In [14]:
%pxresult_first

start 1
time.struct_time(tm_year=2020, tm_mon=5, tm_mday=25, tm_hour=16, tm_min=29, tm_sec=16, tm_wday=0, tm_yday=146, tm_isdst=0)


In [13]:
%%px_last --targets 4 --noblock
print('start 2')
import time
time.sleep(6)
print(time.localtime())

<AsyncResult: execute>

In [15]:
%pxresult_last

start 2
time.struct_time(tm_year=2020, tm_mon=5, tm_mday=25, tm_hour=16, tm_min=29, tm_sec=14, tm_wday=0, tm_yday=146, tm_isdst=0)


In [ ]:
%px_ %connect_info